# Figure understanding & hierarchical document structure analysis

This notebook demonstrates an example of using Azure AI Document Intelligence to ouptut detected figures and hierarchical document structure (in markdown). It will then crop the figures and send figure content (with its caption) to Azure Open AI GPT-4V model to understand the semantics. The figure description will be used to update the markdown output, which can be further used for [semantic chunking](sample_rag_langchain.ipynb).

![Advanced document insights with figure understanding and hierarchical document structure](https://github.com/microsoft/Form-Recognizer-Toolkit/blob/main/SampleCode/media/figure-understanding.png?raw=true)

## Prerequisites
- An Azure AI Document Intelligence resource in one of the 3 preview regions: **East US**, **West US2**, **West Europe** - follow [this document](https://learn.microsoft.com/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0) to create one if you don't have.
- An Azure AI Search resource - follow [this document](https://learn.microsoft.com/azure/search/search-create-service-portal) to create one if you don't have.
- An Azure OpenAI resource and deployments for embeddings model and chat model - follow [this document](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal) to create one if you don't have.

## Setup

In [ ]:
"""
%%writefile .env

AZURE_OPENAI_ENDPOINT = ""  # Your Azure OpenAI resource's endpoint value.
AZURE_OPENAI_API_KEY = "" # Your Azure OpenAI resource's API key value.
AZURE_SEARCH_ENDPOINT = "" # Your Azure AI Search resource's endpoint value.
AZURE_SEARCH_ADMIN_KEY = "" # Your Azure AI Search resource's admin key value.
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT= "" # Your Azure Document Intelligence resource's endpoint value.
AZURE_DOCUMENT_INTELLIGENCE_KEY = "" # Your Azure Document Intelligence resource's key value.
"""

In [ ]:
%pip install -r requirements.txt

In [4]:
"""
This code loads environment variables using the `dotenv` library and sets the necessary environment variables for Azure services.
The environment variables are loaded from the `.env` file in the same directory as this notebook.
"""

import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat
from openai import AzureOpenAI

load_dotenv()

doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

aoai_api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key= os.getenv("AZURE_OPENAI_API_KEY")
aoai_deployment_name = 'gpt-4o' # your model deployment name for GPT-4o
aoai_api_version = '2024-08-01-preview' # Version specified in Endpoint Target URI

## Crop figure from the document (pdf or image) based on the bounding box

In [2]:
from PIL import Image
import fitz  # PyMuPDF
import mimetypes

def crop_image_from_image(image_path, page_number, bounding_box):
    """
    Crops an image based on a bounding box.

    :param image_path: Path to the image file.
    :param page_number: The page number of the image to crop (for TIFF format).
    :param bounding_box: A tuple of (left, upper, right, lower) coordinates for the bounding box.
    :return: A cropped image.
    :rtype: PIL.Image.Image
    """
    with Image.open(image_path) as img:
        if img.format == "TIFF":
            # Open the TIFF image
            img.seek(page_number)
            img = img.copy()
            
        # The bounding box is expected to be in the format (left, upper, right, lower).
        cropped_image = img.crop(bounding_box)
        return cropped_image

def crop_image_from_pdf_page(pdf_path, page_number, bounding_box):
    """
    Crops a region from a given page in a PDF and returns it as an image.

    :param pdf_path: Path to the PDF file.
    :param page_number: The page number to crop from (0-indexed).
    :param bounding_box: A tuple of (x0, y0, x1, y1) coordinates for the bounding box.
    :return: A PIL Image of the cropped area.
    """
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_number)
    
    # Cropping the page. The rect requires the coordinates in the format (x0, y0, x1, y1).
    bbx = [x * 72 for x in bounding_box]
    rect = fitz.Rect(bbx)
    pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72), clip=rect)
    
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    
    doc.close()

    return img

def crop_image_from_file(file_path, page_number, bounding_box):
    """
    Crop an image from a file.

    Args:
        file_path (str): The path to the file.
        page_number (int): The page number (for PDF and TIFF files, 0-indexed).
        bounding_box (tuple): The bounding box coordinates in the format (x0, y0, x1, y1).

    Returns:
        A PIL Image of the cropped area.
    """
    mime_type = mimetypes.guess_type(file_path)[0]
    
    if mime_type == "application/pdf":
        return crop_image_from_pdf_page(file_path, page_number, bounding_box)
    else:
        return crop_image_from_image(file_path, page_number, bounding_box)


## Use Azure OpenAI (GPT-4V model) to understand the semantics of the figure content

In [3]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


In [4]:
MAX_TOKENS = 2000

def understand_image_with_gptv(api_base, api_key, deployment_name, api_version, image_path, caption):
    """
    Generates a description for an image using the GPT-4V model.

    Parameters:
    - api_base (str): The base URL of the API.
    - api_key (str): The API key for authentication.
    - deployment_name (str): The name of the deployment.
    - api_version (str): The version of the API.
    - image_path (str): The path to the image file.
    - caption (str): The caption for the image.

    Returns:
    - img_description (str): The generated description for the image.
    """
    client = AzureOpenAI(
        api_key=api_key,  
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{deployment_name}"
    )

    data_url = local_image_to_data_url(image_path)

    # We send both image caption and the image body to GPTv for better understanding
    if caption != "":
        response = client.chat.completions.create(
                model=deployment_name,
                messages=[
                    { "role": "system", "content": "You are an assisstant embedded in Hyundai vehicle." },
                    { "role": "user", "content": [  
                        { 
                            "type": "text", 
                            "text": f"Describe this image (note: it has image caption: {caption}):" 
                        },
                        { 
                            "type": "image_url",
                            "image_url": {
                                "url": data_url
                            }
                        }
                    ] } 
                ],
                max_tokens=MAX_TOKENS
            )

    else:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                { "role": "system", "content": "You are an assisstant embedded in Hyundai vehicle." },
                { "role": "user", "content": [  
                    { 
                        "type": "text", 
                        "text": "Describe this image:" 
                    },
                    { 
                        "type": "image_url",
                        "image_url": {
                            "url": data_url
                        }
                    }
                ] } 
            ],
            max_tokens=MAX_TOKENS
        )

    img_description = response.choices[0].message.content
    
    return img_description

## Update markdown figure content section with the description from GPT-4V model

In [5]:
def update_figure_description(md_content, img_description, idx):
    """
    Updates the figure description in the Markdown content.

    Args:
        md_content (str): The original Markdown content.
        img_description (str): The new description for the image.
        idx (int): The index of the figure.

    Returns:
        str: The updated Markdown content with the new figure description.
    """

    # The substring you're looking for
    start_substring = f"![](figures/{idx})"
    end_substring = "</figure>"
    new_string = f"<!-- FigureContent=\"{img_description}\" -->"
    
    new_md_content = md_content
    # Find the start and end indices of the part to replace
    start_index = md_content.find(start_substring)
    if start_index != -1:  # if start_substring is found
        start_index += len(start_substring)  # move the index to the end of start_substring
        end_index = md_content.find(end_substring, start_index)
        if end_index != -1:  # if end_substring is found
            # Replace the old string with the new string
            new_md_content = md_content[:start_index] + new_string + md_content[end_index:]
    
    return new_md_content


## Analyze a document with Azure AI Document Intelligence Layout model and update figure description in the markdown output

In [6]:

def analyze_layout(input_file_path, output_folder):
    """
    Analyzes the layout of a document and extracts figures along with their descriptions, then update the markdown output with the new description.

    Args:
        input_file_path (str): The path to the input document file.
        output_folder (str): The path to the output folder where the cropped images will be saved.

    Returns:
        str: The updated Markdown content with figure descriptions.

    """
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=doc_intelligence_endpoint, 
        credential=AzureKeyCredential(doc_intelligence_key),
        headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
    )

    with open(input_file_path, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", output_content_format=ContentFormat.MARKDOWN 
        )

    result = poller.result()
    md_content = result.content
    
    
    if result.figures:
        print("Figures:")
        for idx, figure in enumerate(result.figures):
            figure_content = ""
            img_description = ""
            print(f"Figure #{idx} has the following spans: {figure.spans}")
            for i, span in enumerate(figure.spans):
                print(f"Span #{i}: {span}")
                figure_content += md_content[span.offset:span.offset + span.length]
            print(f"Original figure content in markdown: {figure_content}")

            # Note: figure bounding regions currently contain both the bounding region of figure caption and figure body
            if figure.caption:
                caption_region = figure.caption.bounding_regions
                print(f"\tCaption: {figure.caption.content}")
                print(f"\tCaption bounding region: {caption_region}")
                for region in figure.bounding_regions:
                    if region not in caption_region:
                        print(f"\tFigure body bounding regions: {region}")
                        # To learn more about bounding regions, see https://aka.ms/bounding-region
                        boundingbox = (
                                region.polygon[0],  # x0 (left)
                                region.polygon[1],  # y0 (top)
                                region.polygon[4],  # x1 (right)
                                region.polygon[5]   # y1 (bottom)
                            )
                        print(f"\tFigure body bounding box in (x0, y0, x1, y1): {boundingbox}")
                        cropped_image = crop_image_from_file(input_file_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                        # Get the base name of the file
                        base_name = os.path.basename(input_file_path)
                        # Remove the file extension
                        file_name_without_extension = os.path.splitext(base_name)[0]

                        output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                        cropped_image_filename = os.path.join(output_folder, output_file)

                        cropped_image.save(cropped_image_filename)
                        print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")
                        img_description += understand_image_with_gptv(aoai_api_base, aoai_api_key, aoai_deployment_name, aoai_api_version, cropped_image_filename, figure.caption.content)
                        print(f"\tDescription of figure {idx}: {img_description}")
            else:
                print("\tNo caption found for this figure.")
                for region in figure.bounding_regions:
                    print(f"\tFigure body bounding regions: {region}")
                    # To learn more about bounding regions, see https://aka.ms/bounding-region
                    boundingbox = (
                            region.polygon[0],  # x0 (left)
                            region.polygon[1],  # y0 (top
                            region.polygon[4],  # x1 (right)
                            region.polygon[5]   # y1 (bottom)
                        )
                    print(f"\tFigure body bounding box in (x0, y0, x1, y1): {boundingbox}")

                    cropped_image = crop_image_from_file(input_file_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                    # Get the base name of the file
                    base_name = os.path.basename(input_file_path)
                    # Remove the file extension
                    file_name_without_extension = os.path.splitext(base_name)[0]

                    output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                    cropped_image_filename = os.path.join(output_folder, output_file)
                    # cropped_image_filename = f"data/cropped/image_{idx}.png"
                    cropped_image.save(cropped_image_filename)
                    print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")
                    img_description += understand_image_with_gptv(aoai_api_base, aoai_api_key, aoai_deployment_name, aoai_api_version, cropped_image_filename, "")
                    print(f"\tDescription of figure {idx}: {img_description}")
            
            # replace_figure_description(figure_content, img_description, idx)
            md_content = update_figure_description(md_content, img_description, idx)

    return md_content
            


In [7]:
import fitz

def extract_pages(input_pdf, output_pdf, start_page, end_page):
    doc = fitz.open(input_pdf)
    new_doc = fitz.open()

    for i in range(start_page - 1, end_page):  # Pages are 0-indexed in PyMuPDF
        new_doc.insert_pdf(doc, from_page=i, to_page=i)
    
    new_doc.save(output_pdf)
    print(f"Extracted pages saved as {output_pdf}")

# Example usage: Extract pages 10-15
extract_pages("data/Owner's_Manual.pdf", "data/manual_sample.pdf", 160, 170)

Extracted pages saved as data/manual_sample.pdf


In [8]:
updated_md_with_figure_understanding = analyze_layout("data/manual_sample.pdf", "data/cropped")

print("-------------------------------------------------------------------------------------------")
print(f"Updated markdown content with figure understanding: {updated_md_with_figure_understanding}")


Figures:
Figure #0 has the following spans: [{'offset': 227, 'length': 18}]
Span #0: {'offset': 227, 'length': 18}
Original figure content in markdown: <figure>
</figure>
	No caption found for this figure.
	Figure body bounding regions: {'pageNumber': 1, 'polygon': [2.3978, 2.3738, 2.9898, 2.3738, 2.9899, 2.8101, 2.3979, 2.8101]}
	Figure body bounding box in (x0, y0, x1, y1): (2.3978, 2.3738, 2.9899, 2.8101)
	Figure 0 cropped and saved as data/cropped\manual_sample_cropped_image_0.png
	Description of figure 0: This image is a warning symbol that indicates an airbag, commonly found in vehicle dashboards. It shows a figure of a person with an inflated airbag deploying in front of them, symbolizing the airbag safety system. It serves as a reminder or warning related to airbags, such as a malfunction or system status.
Figure #1 has the following spans: [{'offset': 609, 'length': 18}]
Span #0: {'offset': 609, 'length': 18}
Original figure content in markdown: <figure>
</figure>
	No caption 

In [9]:
# Save as markdown file
with open("data/analyzed_results.md", "w", encoding="utf-8") as file:
    file.write(updated_md_with_figure_understanding)

print("Markdown file saved as data/analyzed_results.md")


Markdown file saved as data/analyzed_results.md


# Evaluate

In [5]:
import pandas as pd
import os
import json

from pprint import pprint
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator
from azure.ai.evaluation import GroundednessEvaluator, GroundednessProEvaluator
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    Dataset,
    EvaluatorConfiguration,
    ConnectionType,
    EvaluationSchedule,
    RecurrenceTrigger,
    ApplicationInsightsConfiguration
)
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
    F1ScoreEvaluator,
    RetrievalEvaluator
)
# from model_endpoint import ModelEndpoint



load_dotenv(override=True)

True

In [ ]:
credential = DefaultAzureCredential()

# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai",
}


In [7]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ.get("AZURE_AI_PROJECT_CONN_STR"),  # At the moment, it should be in the format "<Region>.api.azureml.ms;<AzureSubscriptionId>;<ResourceGroup>;<HubName>" Ex: eastus2.api.azureml.ms;xxxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxxxxx;rg-sample;sample-project-eastus2
)

### Save Doc-Int result md as vector

In [ ]:
# pass

### Evaluate

Doc-based response eval score vs MD-based response eval score (vs Intent-RAG or Intent-BingSearch ... Routing)

In [ ]:
updated_md_with_figure_understanding = []
with open("data/analyzed_results.md", "r", encoding="utf-8") as file:
    for line in file:
        updated_md_with_figure_understanding.append(line.strip())  # Strip removes extra spaces or newlines

print("\n".join(updated_md_with_figure_understanding))  # Join lines into a full string


In [14]:
client = AzureOpenAI(
    api_key=model_config['api_key'],  
    api_version=model_config['api_version'],
    base_url=f"{model_config['azure_endpoint']}/openai/deployments/{model_config['azure_deployment']}"
)

def ask_question(query):
    MAX_TOKENS = 2000

    response = client.chat.completions.create(
        model=model_config['azure_deployment'],
        messages=[
            {"role": "system", "content": 
             """
             You are an assisstant embedded in Hyundai vehicle.
             Using information from the given context, create responses about the following request.
             Specify the reference.
            - Questions about image alerts
            - Questions about reconfiguring vehicle after alert
            - Additional user tips
            """},
            {"role": "user", "content": f"Context: {updated_md_with_figure_understanding}"},
            { "role": "user", "content": query}
        ],
        max_tokens=MAX_TOKENS
        )
    response_parsed = response.choices[0].message.content
    
    print(response_parsed)
    # return response_parsed



In [15]:
ask_question("I have a mark with plus and minus sign on my dashboard")

Based on your description, it sounds like the **Charging System Warning Light** is illuminated. This warning light typically features a "+" and "-" symbol and indicates an issue with your vehicle's alternator or electrical charging system.

### What to do:
1. **Stop the Vehicle Safely:**
   - Drive carefully to the nearest safe location and stop your vehicle.

2. **Check the Alternator Drive Belt:**
   - Turn off the engine and inspect the alternator drive belt for signs of looseness or breakage.

3. **If the Belt Looks Normal:**
   - There may be a problem with the electrical charging system itself.

4. **Next Action:**
   - In both cases, **Hyundai recommends having your vehicle inspected by an authorized Hyundai dealer as soon as possible** to prevent further issues or a potential loss of battery power.

### Additional Tip:
Avoid driving the vehicle for extended periods with this warning light illuminated, as it can lead to the battery draining completely and can strand you unexpect